# Comp_Finance

In [59]:
!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install seaborn
!pip install scikit-learn

In [60]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [63]:
# Core Stock API(Time_Series_Daily)

import requests

r = requests.get('https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=IBM&apikey=P5ZI3O6DR29N16M6')
data = r.json()

print(data)

{'Information': 'Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}


In [62]:
time_series_data = data['Time Series (Daily)']

df = pd.DataFrame.from_dict(time_series_data, orient='index')

df.index = pd.to_datetime(df.index)

df.columns = ['open', 'high', 'low', 'close', 'volume']

df = df.astype(float)

print(df.head(50))


KeyError: 'Time Series (Daily)'

In [ ]:
import requests
import pandas as pd

api_key = 'P5ZI3O6DR29N16M6'

symbol = 'IBM'
interval = 'daily'
time_period = 10
series_type = 'close'

url = f'https://www.alphavantage.co/query?function=SMA&symbol={symbol}&interval={interval}&time_period={time_period}&series_type={series_type}&apikey={api_key}'
response = requests.get(url)
sma_data = response.json()

import json
print(json.dumps(sma_data, indent=4))


In [ ]:
sma_series = sma_data['Technical Analysis: SMA']

sma_df = pd.DataFrame.from_dict(sma_series, orient='index')

sma_df.index = pd.to_datetime(sma_df.index)

sma_df.columns = ['SMA']

sma_df = sma_df.astype(float)

print(sma_df.head(30))


In [ ]:
merged_df = df.join(sma_df, how='inner')

print(merged_df.head(30))


In [64]:
url = f'https://www.alphavantage.co/query?function=RSI&symbol={symbol}&interval={interval}&time_period=14&series_type={series_type}&apikey={api_key}'
response = requests.get(url)
rsi_data = response.json()

print(json.dumps(rsi_data, indent=4))

rsi_series = rsi_data['Technical Analysis: RSI']

rsi_df = pd.DataFrame.from_dict(rsi_series, orient='index')

rsi_df.index = pd.to_datetime(rsi_df.index)

rsi_df.columns = ['RSI']

rsi_df = rsi_df.astype(float)

print(rsi_df.head())

merged_df = merged_df.join(rsi_df, how='inner')

print(merged_df.head())


{
    "Information": "Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits."
}


KeyError: 'Technical Analysis: RSI'

In [ ]:
import requests
import pandas as pd

api_key = 'YOUR_API_KEY'
symbol = 'IBM'

url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&symbol={symbol}&apikey={api_key}'
response = requests.get(url)
news_data = response.json()

import json
print(json.dumps(news_data, indent=4))


In [ ]:
from textblob import TextBlob

articles = news_data['feed']

articles_df = pd.DataFrame(articles)

def get_sentiment(text):
    analysis = TextBlob(text)
    return analysis.sentiment.polarity

articles_df['sentiment'] = articles_df['summary'].apply(get_sentiment)

sentiment_by_date = articles_df.groupby('published_date')['sentiment'].mean()

print(sentiment_by_date.head())


In [ ]:
merged_df = merged_df.join(sentiment_by_date, how='left')

merged_df['signal'] = 0
merged_df.loc[(merged_df['close'] > merged_df['SMA']) & (merged_df['sentiment'] > 0), 'signal'] = 1
merged_df.loc[(merged_df['close'] < merged_df['SMA']) | (merged_df['sentiment'] < 0), 'signal'] = -1

print(merged_df.head())


In [ ]:
initial_cash = 100000
position = 0
cash = initial_cash
positions = []

for index, row in merged_df.iterrows():
    if row['signal'] == 1 and position == 0:
        position = cash / row['close']
        cash = 0
        positions.append((index, 'Buy', row['close']))
    elif row['signal'] == -1 and position > 0:
        cash = position * row['close']
        position = 0
        positions.append((index, 'Sell', row['close']))

final_value = cash + position * merged_df.iloc[-1]['close']
print(f'Final portfolio value: {final_value}')


In [ ]:
import numpy as np

portfolio = pd.DataFrame(index=merged_df.index)
portfolio['holdings'] = merged_df['close'] * position
portfolio['cash'] = cash
portfolio['total'] = portfolio['holdings'] + portfolio['cash']

portfolio['returns'] = portfolio['total'].pct_change()

cumulative_returns = portfolio['total'][-1] / portfolio['total'][0] - 1
sharpe_ratio = np.sqrt(252) * (portfolio['returns'].mean() / portfolio['returns'].std())
drawdown = (portfolio['total'].cummax() - portfolio['total']) / portfolio['total'].cummax()

print(f'Cumulative Returns: {cumulative_returns}')
print(f'Sharpe Ratio: {sharpe_ratio}')
print(f'Max Drawdown: {drawdown.max()}')


In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

sma_periods = [10, 20, 50]

tscv = TimeSeriesSplit(n_splits=5)
for train_index, test_index in tscv.split(merged_df):
    train, test = merged_df.iloc[train_index], merged_df.iloc[test_index]
    for period in sma_periods:
        train['SMA'] = train['close'].rolling(window=period).mean()
        test['SMA'] = test['close'].rolling(window=period).mean()
